# Assignment 3 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Personal Details:

In [4]:
# Details Student 1:
# Lior Fridman 206798902 computer-science
# Details Student 2:


## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [5]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [6]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [7]:
# word net installation:

# unmark if you want to use and need to install
# !pip install wn
# !python -m wn download omw-he:1.4

In [8]:
# word net import:

# unmark if you want to use:
# import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [9]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
# !pip install hebrew_tokenizer

In [10]:
# Hebrew tokenizer import:

# unmark if you want to use:
import hebrew_tokenizer as ht

c:\Users\liors\Documents\מדעי המחשב שנה ג\machineLearning\assignment3-text_analysis


### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [11]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [12]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [13]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

In [25]:
from sklearn.metrics import make_scorer
from sklearn.feature_selection import SelectKBest, chi2

# hebrew tokenizer function
def tokenizer(text):
    tokens = ht.tokenize(text)
    tokens_list = []
    for grp, token, token_num, (start_index, end_index) in tokens:
        if grp == 'HEBREW':
            tokens_list.append(token)
    return tokens_list
    
# Hyperparameter tuning parameters
TUNING_PARAMETERS = {
    'Decision Tree': {
        'classifier': DecisionTreeClassifier(),
        'params': {
            'criterion': ['gini', 'entropy'],
            'max_depth': [10, 20],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        }
    },
    'Linear Support Vector Machine': {
        'classifier': LinearSVC(),
        'params': {
            'C': [0.1, 1, 10],
            'max_iter': [1000, 2000, 3000]
        }
    },
        'KNN': {
        'classifier': KNeighborsClassifier(),
        'params': {
            'n_neighbors': [3, 5, 7],
            'p': [1, 2]
        }
    },
}

def find_best_model(X_train_dense, y_train):
    best_model = None
    best_model_name = ""
    best_avg_f1 = 0
    
    cv = 10 
    
    models = {
        'Decision Tree': DecisionTreeClassifier(),
        'Linear Support Vector Machine': LinearSVC(),
        'SGD Classifier': SGDClassifier(),
        'KNN': KNeighborsClassifier(), 
    }

    for model_name, model in models.items():
        # fit the model
        model.fit(X_train_dense, y_train)

        if model_name in TUNING_PARAMETERS:
            classifier = TUNING_PARAMETERS[model_name]['classifier']
            params = TUNING_PARAMETERS[model_name]['params']

            cls_cv = GridSearchCV(classifier, params, cv=cv, scoring='f1_macro')

            # fit GridSearchCV on the models
            grid_search_res = cls_cv.fit(X_train_dense, y_train)

            best_params_model = grid_search_res.best_estimator_
            
        else:
            best_params_model = model

        # calculate f1 scores using cross-validation
        f1_scores = cross_val_score(best_params_model, X_train_dense, y_train, scoring=make_scorer(f1_score, average='macro'), cv=10)
        average_f1 = f1_scores.mean()

        if average_f1 > best_avg_f1:
            best_avg_f1 = average_f1
            best_model = best_params_model
            best_model_name = model_name

    print('----------------------------------------------------')
    return best_model, best_model_name, best_avg_f1

In [22]:
# initializing a flag to track if a good model was found
good_model_found = False

# iterate the find_best_model function until a good model is found
while not good_model_found:
    # initializing the vectorizer
    tfidf_vectorizer = TfidfVectorizer(tokenizer=tokenizer, ngram_range=(1, 3))

    # fit and transform the data using TF-IDF
    X_train_tfidf = tfidf_vectorizer.fit_transform(df_train['story'])
    y_train = df_train['gender']
    X_test_tfidf = tfidf_vectorizer.transform(df_test['story'])

    # feature selection
    k_best = SelectKBest(score_func=chi2, k=100)
    X_train_selected = k_best.fit_transform(X_train_tfidf, y_train)
    X_test_selected = k_best.transform(X_test_tfidf)

    # Convert sparse matrix to dense array
    X_train_selected_dense = X_train_selected.toarray()
    X_test_selected_dense = X_test_selected.toarray()

    # PCA on the training data
    pca = PCA(n_components=99)
    X_train_pca = pca.fit_transform(X_train_selected_dense)
    X_test_pca = pca.transform(X_test_selected_dense)

    best_model, best_model_name, best_avg_f1 = find_best_model(X_train_pca, y_train)
    
    print(f"Average F1 score for {best_model_name}: {best_avg_f1}")
    
    if best_avg_f1 >= 0.7:
        good_model_found = True
        print(f"Found a good model: {best_model_name} with an average F1 score of {best_avg_f1}")
    else:
        print(f"The model {best_model_name} did not meet the criteria, trying again...")

    # Get the feature names from the TF-IDF vectorizer
    selected_feature_indices = k_best.get_support(indices=True)
    feature_names = tfidf_vectorizer.get_feature_names_out()
    selected_tokens = [feature_names[idx] for idx in selected_feature_indices]
    
print('----------------------------------------------------')

# prediction of results using PCA data
predicted_gender = best_model.predict(X_test_pca)

# print the tokens from the selected k best features
print("Tokens of selected k best features:", selected_tokens)


----------------------------------------------------
Average F1 score for Decision Tree: 0.8269943204302541
Found a good model: Decision Tree with an average F1 score of 0.8269943204302541
----------------------------------------------------
Tokens of selected k best features: ['אוהבת', 'איטלקית', 'איש בשטח', 'איש בשטח סגור', 'אני זוכרת', 'אני חושבת', 'אני חייבת', 'אני יודעת', 'אני יכולה', 'אני עובדת', 'אני שמחה', 'את האירוע', 'ביום השני הציעו', 'בכלב', 'בן הזוג', 'בן הזוג שלי', 'בן זוגי', 'בסרט אימה', 'בשטח סגור', "ג'וני", 'דגל', 'דייטים', 'האקתון', 'הבן זוג', 'הגורה', 'הדגל', 'הולכת', 'החתן', 'היה ממש יפה', 'הייתי בטוחה', 'הייתי צריכה', 'המורים', 'המצגת', 'העסק', 'הפנגויה', 'הציעו לי', 'הציעו לי נישואים', 'הרגשתי כמו בסרט', 'הריצוף', 'השטיפה', 'השני הציעו', 'השני הציעו לי', 'ובעלי', 'והריצוף', 'ועד הבית', 'ותמרה', 'זוגי', 'זוכרת', 'זוכרת את', 'חברות', 'חברות שלי', 'חופה', 'חופה מצומצמת', 'חושבת', 'חייבת', 'טו', 'טו טו', 'טירן', 'טסתי לתאילנד', 'יודעת', 'יכולה', 'ילדה', 'ילדה קטנה', '

In [23]:
# creating a DataFrame for the predictions
df_predicted = pd.DataFrame({
    'test_example_id': df_test.index,
    'story': df_test['story'],
    'gender': predicted_gender
})

# display the predictions for the first and last 5 rows
print("Predictions for the first 5 examples:")
print(df_predicted.head())

print("\nPredictions for the last 5 examples:")
print(df_predicted.tail())

Predictions for the first 5 examples:
   test_example_id                                              story gender
0                0  כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...      m
1                1  הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת "...      m
2                2  אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...      f
3                3  רגע הגיוס לצבא היה הרגע הכי משמעותי עבורי, אני...      m
4                4  אני הגעתי לברזיל ישר מקולומביה וגם אני עשיתי ע...      m

Predictions for the last 5 examples:
     test_example_id                                              story gender
318              318  בשנה האחרונה הרגשתי די תקוע בעבודה, השגרה הפכה...      m
319              319  אני ואילן חברים טובים מזה 20 שנה תמיד חלמנו לפ...      f
320              320  מידי יום שישי אני נוהג לנסוע בתחבורה ציבורית ס...      f
321              321  לפני מספר חודשים, בשיא התחלואה של הגל השני, עמ...      m
322              322  היום בו דיווחתי על גניבה של האוטו שלי. בוקר א

### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [24]:
df_predicted.to_csv('classification_results.csv',index=False)